In [2]:
import pandas as pd
import numpy as np
from time import sleep
import jellyfish as jf
import json

In [3]:
data = pd.read_pickle(r'full_data_unvalidated_wip.pkl')
data = data.astype({'kinopoiskid': 'int32'})

In [4]:
pd.set_option('display.max_rows', 300)
data.head(1000)

,actor,age_restriction_aliases,age_access_type,country,box_office_worldwide,average_rating,box_office_russia,duration,type,name,...,release_type,release_year,budget,digital_uid,genre,director,kinopoiskid,matched_by,match_coeff,full_info
unified_element_uid,,,,,,,,,,,,,,,,,,,,,
e785baa6-f175-42b4-9e16-4319ac7991d5,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, 21c7a45...",[CR_9_13],12,[usa],-1,6.21,-1,6420000,MOVIE,Звёздный путь 5: Последний рубеж,...,RELEASE,1989,27800000,2870,"[Sci-Fi, Action, Thriller, Adventure]",[cff3362f-ad91-498d-80b3-ea3ab8ec65cf],6580,okko_ru,2.000000,"{""kinopoisk_id"": 6580, ""imdb_id"": ""tt0098382"",..."
4593737e-de9c-40df-97db-fb3cf85a08ef,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, 21c7a45...",-1,16,[usa],-1,7.20,-1,6780000,MOVIE,Звёздный путь 6: Неоткрытая страна,...,RELEASE,1991,30000000,2873,"[Sci-Fi, Action, Thriller, Adventure]",[bab6b7f4-4506-458f-9091-d567505674f2],6460,okko_ru,2.000000,"{""kinopoisk_id"": 6460, ""imdb_id"": ""tt0102975"",..."
11ba66db-e941-4c3a-8da6-d8900e56f8c7,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, bcad17e...",-1,18,[usa],-1,4.00,-1,6000000,MOVIE,В поисках древнего артефакта,...,RELEASE,2019,-1,52428009,[Horror],[895f5774-964a-4c5a-ae22-d697d3e3e620],1228112,alias,1.538957,"{""kinopoisk_id"": 1228112, ""imdb_id"": ""tt946984..."
3f30a2ef-53b7-40e3-954f-1bdfc38a6d17,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, 30dd19c...",-1,18,[usa],-1,6.99,-1,3780000,MOVIE,Прожарка Чарли Шина,...,RELEASE,2011,-1,1084933,"[Comedy, Documentary]",[1de22aff-430e-4af9-bf1d-159dbf8e9269],623934,okko_ru,2.000000,"{""kinopoisk_id"": 623934, ""imdb_id"": ""tt1985970..."
cdfa700f-122d-41e5-b8dc-9c6813bab6d2,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, a805fdd...",-1,18,[usa],-1,6.00,-1,3780000,MOVIE,Прожарка Уильяма Шэтнера,...,RELEASE,2006,-1,1191553,[Comedy],[1de22aff-430e-4af9-bf1d-159dbf8e9269],893637,okko_ru,2.000000,"{""kinopoisk_id"": 893637, ""imdb_id"": ""tt0840305..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
facbf429-9c5f-4d00-89e1-52f6b7a3026e,"[5b42d9d3-3301-491e-904d-3ddc0b6f29af, e5489b9...","[CR_5_8, CR_9_13]",6,[usa],-1,6.00,-1,4680000,MOVIE,Джейк и Гиганты,...,RELEASE,2015,-1,68866,"[ForKids, 287a1485-7a88-4c2f-bc94-ca418b6c47a1]",[50882502-a0d2-4aaf-8b91-80597b60ed3a],911598,okko_ru,2.000000,NaN
010cdeb4-f10d-42c8-91eb-c31600eb3083,"[806640dc-f610-41bb-be44-5d3d06e563ba, 2ab1682...",-1,18,"[usa, mexico]",-1,5.45,-1,6180000,MOVIE,Судная ночь навсегда,...,RELEASE,2021,-1,726288,"[Action, Thriller, Horror]",[45714a7b-5d0a-4a2e-b936-f2b3f6fd5973],1263408,okko_ru,2.000000,NaN
be225380-dfbf-4d0c-a962-120ffba50a63,"[74184166-07ff-43a4-9df9-9dcab7f2c13d, 1830426...",-1,18,"[usa, great-britain, canada, sweden]",-1,5.76,-1,6300000,MOVIE,Убийца внутри меня,...,RELEASE,2010,13000000,34994,"[Thriller, Drama, 364fdc2e-bdfe-40be-b2c5-d30f...",[8bac9f07-de85-450a-9c3e-84d697153bbc],431294,okko_ru,2.000000,NaN


In [129]:
full_records, partial_records = [x.reset_index() for _, x in data.groupby(data['kinopoiskid'] < 0)]

In [ ]:
failed, mismatched  = [x.reset_index() for _, x in partial_records.groupby(partial_records['matched_by'] == 'year_mismatch')]

In [130]:
full_records

,unified_element_uid,actor,age_restriction_aliases,age_access_type,country,box_office_worldwide,average_rating,box_office_russia,duration,type,...,alias,release_type,release_year,budget,digital_uid,genre,director,kinopoiskid,matched_by,match_coeff
0,e785baa6-f175-42b4-9e16-4319ac7991d5,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, 21c7a45...",[CR_9_13],12,[usa],-1,6.21,-1,6420000,MOVIE,...,star-trek-v-the-final-frontier,RELEASE,1989,27800000,2870,"[Sci-Fi, Action, Thriller, Adventure]",[cff3362f-ad91-498d-80b3-ea3ab8ec65cf],6580,okko_ru,2.000000
1,4593737e-de9c-40df-97db-fb3cf85a08ef,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, 21c7a45...",-1,16,[usa],-1,7.20,-1,6780000,MOVIE,...,star-trek-vi-the-undiscovered-country,RELEASE,1991,30000000,2873,"[Sci-Fi, Action, Thriller, Adventure]",[bab6b7f4-4506-458f-9091-d567505674f2],6460,okko_ru,2.000000
2,11ba66db-e941-4c3a-8da6-d8900e56f8c7,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, bcad17e...",-1,18,[usa],-1,4.00,-1,6000000,MOVIE,...,devils-revenge,RELEASE,2019,-1,52428009,[Horror],[895f5774-964a-4c5a-ae22-d697d3e3e620],1228112,alias,1.538957
3,3f30a2ef-53b7-40e3-954f-1bdfc38a6d17,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, 30dd19c...",-1,18,[usa],-1,6.99,-1,3780000,MOVIE,...,the-roast-of-charlie-sheen,RELEASE,2011,-1,1084933,"[Comedy, Documentary]",[1de22aff-430e-4af9-bf1d-159dbf8e9269],623934,okko_ru,2.000000
4,cdfa700f-122d-41e5-b8dc-9c6813bab6d2,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, a805fdd...",-1,18,[usa],-1,6.00,-1,3780000,MOVIE,...,the-roast-of-william-shatner,RELEASE,2006,-1,1191553,[Comedy],[1de22aff-430e-4af9-bf1d-159dbf8e9269],893637,okko_ru,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10914,43bddcc0-db12-428e-9c65-f5606aa4b989,[d08bba89-e937-40f3-b2a7-7ea26de4c246],-1,0,[russia],-1,-1.00,-1,25140000,SERIAL,...,zagadki-russkojj-istorii,RELEASE,2011,-1,368638083,"[Documentary, d7214feb-8c11-4aea-aabb-ac98a8d5...",[384deeb0-ab24-4e3f-9744-1660b2ecaf67],718006,okko_ru,1.000000
10915,ac0c9152-c070-43a0-bb99-7e72764b7c0a,[d08bba89-e937-40f3-b2a7-7ea26de4c246],"[CR_0_4, CR_5_8]",0,[brasil],-1,-1.00,-1,5400000,SERIAL,...,inventing-games,RELEASE,2019,-1,388072866,[287a1485-7a88-4c2f-bc94-ca418b6c47a1],[ecc88cca-7660-4d74-a0af-18ca7069a743],4950630,okko_ru,2.000000
10916,ce8d2906-a33f-4140-9ffe-e091990d71a4,[50ed4519-ad24-47b3-b135-947dba796372],"[CR_5_8, CR_9_13]",6,[russia],-1,-1.00,-1,3600000,SERIAL,...,bolshie-igry,RELEASE,2022,-1,376604149,"[ForKids, 1f22ccf1-288a-4e6e-b39a-7502799e7125]",[ad784ddb-a602-4082-bf6f-f83ec5a96edf],976636,okko_ru,2.000000
10917,0bfb5a13-50fc-48f3-b51a-a30971024651,"[d658fbcd-8e80-4880-a25c-e7a209998969, b6fe1a3...","[CR_0_4, CR_0_13, CR_5_8, CR_9_13]",0,[china],-1,-1.00,-1,40560000,SERIAL,...,environmental-task-force,RELEASE,2021,-1,375536499,[287a1485-7a88-4c2f-bc94-ca418b6c47a1],[9d6f5496-2d1b-4f21-a2d8-52b316295c4a],5086549,okko_ru,2.000000


In [5]:
from kinopoisk_unofficial.kinopoisk_api_client import KinopoiskApiClient
api_client = KinopoiskApiClient("6a7d2845-12e6-4514-bf86-bf3f892867fd")

In [138]:
from kinopoisk_unofficial.request.films.search_by_keyword_request import SearchByKeywordRequest

def compare_names(target_names, match_names):
#     ru_t, ru_m = target_names['ru'], match_names['ru']
#     en_t, en_m = target_names['en'], match_names['en']
#     print(f'RU names: {ru_t}, {str(ru_m).lower()}')
#     print(f'EN names: {en_t}, {str(en_m).lower()}')
    ru_match = jf.jaro_winkler_similarity(target_names['ru'], str(match_names['ru']).lower())
    en_match = jf.jaro_winkler_similarity(target_names['en'], str(match_names['en']).lower())
    
    return ru_match + en_match

def match_title(okko_name, okko_alias, release_year):
        name_match = api_client.films.send_search_by_keyword_request(SearchByKeywordRequest(okko_name))
        alias_match = api_client.films.send_search_by_keyword_request(SearchByKeywordRequest(okko_alias))
        
        ru_film = name_match.films[0] if name_match.films else None
        en_film = alias_match.films[0] if alias_match.films else None
        if not (ru_film or en_film):
            return -1, 'not found', 0.0
#         if name_match.films[0].film_id == alias_match.films[0].film_id:
#             return name_match.films[0].film_id, 'okko_ru', 2.0
        else:
            okko_keys = {"ru": okko_name, "en": okko_alias}
            name_match_keys = {"ru": ru_film.name_ru if ru_film else '', 
                               "en": ru_film.name_en if ru_film else ''}
            alias_match_keys = {"ru": en_film.name_ru if en_film else '', 
                                "en": en_film.name_en if en_film else ''}
            match_years = {"ru": int(ru_film.year) if ru_film and ru_film.year != 'null' else 0, 
                           "en": int(en_film.year) if en_film and en_film.year != 'null' else 0}
            
#             print(okko_keys)
#             print(name_match_keys)
#             print(alias_match_keys)
#             print(match_years)
#             print(release_year)
            
            name_coeff = compare_names(okko_keys, name_match_keys)
            alias_coeff = compare_names(okko_keys, alias_match_keys)
            
            #print(f'Coeffs: {name_coeff} | {alias_coeff}')
                                        
            if name_coeff > alias_coeff and match_years['ru'] == release_year:
                return name_match.films[0].film_id, 'okko_ru', name_coeff
            elif name_coeff <= alias_coeff and match_years['en'] == release_year:
                return alias_match.films[0].film_id, 'alias', alias_coeff
            else:
                return -1, 'year_mismatch', 0.0

def find_kp_ids(data):
    counter = 0
    for index, row in data[data['matched_by'] == 'year_mismatch'].iterrows():
        kpid, matched_by, coeff = match_title(row['name'].lower(), row['alias'].lower(), row['release_year'])
        
        data.at[index, 'kinopoiskid'] = kpid
        data.at[index, 'matched_by'] = matched_by
        data.at[index, 'match_coeff'] = coeff
        
        counter += 1
        if counter % 100 == 0:
            print(f'Processing record {counter}')

In [137]:
find_kp_ids(data)

AttributeError: 'FoundFilm' object has no attribute 'name_original'

In [121]:
manual_index = '4c4cdf9e-5f19-4588-b99c-aed44acd315c'
manual_kpid = -1

data.at[manual_index, 'kinopoiskid'] = manual_kpid
data.at[manual_index, 'matched_by'] = 'manual_deny'
data.at[manual_index, 'match_coeff'] = 2.0

In [169]:
pd.set_option('display.max_rows', 1000)
data[data['matched_by'] == 'not found'][['name', 'alias', 'release_year']]

,name,alias,release_year
unified_element_uid,,,
b911e591-d2d6-473d-a273-d2f1bb673bc0,Неудержимые 3 (8K),the-expendables-3-8k,2014
9e32305d-d745-4a76-a6e6-76f802c99aa8,Патагония: от Камаронес до горы Дарвина,patagonija-ot-kamarones-do-gory-darvina,2012
96535e44-d8ff-48bf-861d-a46f74a68db8,Лыжная симфония,symphony-on-skis,2017
cdd50eac-dd34-4f47-a43f-aae4465b420d,Фидель Кастро: Жизнь ради революции,fidel-castro-life-for-the-revolution,2014
f25fa3fe-81f3-41ef-8580-7a120bab346e,Невероятная пятёрка. История гепарда,the-incredible-five-a-cheetahs-tale,2019
4c4cdf9e-5f19-4588-b99c-aed44acd315c,Царство разбойника – остров Рождества,realm-of-the-robber-crab-christmas-island,2018
766384cd-e450-4393-b438-14c286806a39,Бетховен. Секретные материалы,the-beethoven-files,2013
5a2c411e-70f9-4a0a-9502-71b567753230,Исландия: На вершине мира,iceland-on-top-of-the-world,2016
98eff766-bc1e-40ba-b10c-b80b8599d953,Уникальная Амазонка: Южная Америка,unikalnaja-amazonka-juzhnaja-amerika,2012


In [179]:
data[data['matched_by'] == 'year_mismatch']

,actor,age_restriction_aliases,age_access_type,country,box_office_worldwide,average_rating,box_office_russia,duration,type,name,alias,release_type,release_year,budget,digital_uid,genre,director,kinopoiskid,matched_by,match_coeff
unified_element_uid,,,,,,,,,,,,,,,,,,,,
d8b55146-dee5-4498-957c-5614414b48fb,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, a51b376...","[CR_0_4, CR_5_8]",0,"[usa, canada]",-1,6.21,-1,4860000,MOVIE,Большое путешествие,the-wild,RELEASE,2006,80000000,52350,"[Family, 287a1485-7a88-4c2f-bc94-ca418b6c47a1]",[fe598c84-ef4f-47c7-91af-d5cbd89c4e8d],-1,year_mismatch,0.0
8889bce7-3110-4851-aba2-5fd51dbd4d9d,"[d28a9a77-7067-465b-8ecc-233824f25e95, abb2e7f...",-1,12,[usa],-1,7.36,-1,5880000,MOVIE,Трое мужчин и младенец,three-men-and-a-baby,RELEASE,1987,11000000,14518,"[Comedy, Drama, 364fdc2e-bdfe-40be-b2c5-d30f43...",[21c7a451-d42c-4c4b-90ee-7bc9658f9361],-1,year_mismatch,0.0
cbadb4e2-6f3c-4b0d-b341-6b9e61101cfe,"[6744ca09-1710-4613-8661-1eb987347505, d0ccef8...",-1,16,[usa],-1,6.00,-1,5820000,MOVIE,Страж Вселенной: петля времени,guardian-of-the-universe,RELEASE,2017,-1,975159,"[Sci-Fi, Action, Thriller, Adventure]",[ee8a3701-9e6e-48fc-816d-16281915a93a],-1,year_mismatch,0.0
8292bb0b-0c2a-4b38-873c-ab6785106d37,"[672b731b-6187-4446-ab20-5129aa6b0f3d, dfd1744...",-1,12,[usa],-1,7.25,-1,8820000,MOVIE,Вестсайдская история,west-side-story-47492936,RELEASE,1961,-1,47492936,"[Melodrama, Drama, Music, 364fdc2e-bdfe-40be-b...","[659b638e-fb54-4c10-9cd0-f2c7652304e8, cfe196a...",-1,year_mismatch,0.0
d927dfa8-af89-4ccd-b941-5d659896c97a,"[69c0257c-6b92-4436-adc3-4327e0743369, 8b115bc...",-1,16,[usa],-1,5.77,-1,5580000,MOVIE,Эон Флакс,aeon-flux,RELEASE,2005,62000000,2526,"[Action, Thriller, Adventure]",[ddad859b-1eed-4c5e-bb52-41a8fa55578b],-1,year_mismatch,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14c1c2f7-e5f9-4f5c-88c5-4a969d5a6677,[08c5c6c3-ec43-486d-9038-188ce809ac38],[CR_0_4],0,[great-britain],-1,-1.00,-1,4500000,SERIAL,Шоу зоопарка,the-zoo-show,RELEASE,2009,-1,375468799,[287a1485-7a88-4c2f-bc94-ca418b6c47a1],[08c5c6c3-ec43-486d-9038-188ce809ac38],-1,year_mismatch,0.0
e8361626-d95e-45df-a2d0-3e0186c058f0,[08c5c6c3-ec43-486d-9038-188ce809ac38],[CR_0_4],0,[great-britain],-1,-1.00,-1,5100000,SERIAL,Шоу на гастролях,the-traveling-show,RELEASE,2020,-1,375468800,[287a1485-7a88-4c2f-bc94-ca418b6c47a1],[08c5c6c3-ec43-486d-9038-188ce809ac38],-1,year_mismatch,0.0
2a9402b6-2c2e-4931-86ea-25712d89981e,[d08bba89-e937-40f3-b2a7-7ea26de4c246],-1,0,[russia],-1,-1.00,-1,25620000,SERIAL,Победы русского оружия,pobedy-russkogo-oruzhija,RELEASE,2016,-1,368638084,"[Documentary, d7214feb-8c11-4aea-aabb-ac98a8d5...",[384deeb0-ab24-4e3f-9744-1660b2ecaf67],-1,year_mismatch,0.0


In [151]:
data[data['type'] == 'MP_MOVIE']

,actor,age_restriction_aliases,age_access_type,country,box_office_worldwide,average_rating,box_office_russia,duration,type,name,alias,release_type,release_year,budget,digital_uid,genre,director,kinopoiskid,matched_by,match_coeff
unified_element_uid,,,,,,,,,,,,,,,,,,,,
43a58d40-4b90-46a8-9672-c534984bc936,"[8cad3a87-66dd-494a-b03d-a82ba4c17537, ef46fb3...",-1,18,"[germany, france, spain, austria]",-1,6.59,-1,10440000,MP_MOVIE,Генрих Наваррский,henri-4,RELEASE,2010,-1,1176124,"[Drama, d7214feb-8c11-4aea-aabb-ac98a8d56fd5]",[03b6f1c9-4090-4aac-980b-38e9da43481f],420660,NaN,NaN
a7731865-7de7-4bd1-b2c7-de1d85ac525d,"[3d6f2a31-5f77-4972-980e-4d662aaf8601, c13971b...",-1,16,"[usa, canada]",-1,6.80,-1,10500000,MP_MOVIE,История Одри Хепберн,the-audrey-hepburn-story,RELEASE,2000,-1,124005,"[Drama, 0ef0aa70-f86c-4141-8054-8b39af97867d]",[17d4ece7-34c6-4336-97e7-16c1390da550],279553,NaN,NaN
13ba3a33-72e6-4810-ace0-24531ebfaafb,[d08bba89-e937-40f3-b2a7-7ea26de4c246],-1,18,"[usa, france, chez, belgium]",-1,6.00,-1,3960000,MP_MOVIE,Annecy Festival 2020. Короткометражные мультфи...,annecy-festival-2020,RELEASE,2021,-1,25516702,[287a1485-7a88-4c2f-bc94-ca418b6c47a1],"[094f3457-6c73-4eb2-b040-3b31bf4e5714, ad65892...",-1,not found,0.000000
b405bd8c-d014-4bc8-81b4-2ffc21862f89,"[dfae7148-84a3-4cc0-910c-8453e4c51ae0, 2bf360b...",-1,12,[usa],-1,7.20,-1,10440000,MP_MOVIE,Настоящая женщина,true-women-505772,RELEASE,1997,-1,505772,"[Action, Melodrama, Drama, Adventure, eb001d27...",[c587dc49-b281-4837-bf19-0184ef530375],89550,NaN,NaN
be0227fc-ad0c-4812-9814-a4c3f1db5563,"[f235f535-b6eb-440c-b7d7-243831835a7c, b625071...",-1,16,"[usa, ussr, czechoslovakia, east-germany]",-1,7.80,-1,11100000,MP_MOVIE,Сталинград,stalingrad-260965876,RELEASE,1989,-1,260965876,"[Drama, d7214feb-8c11-4aea-aabb-ac98a8d56fd5, ...",[2180e108-15d2-4699-b22b-ef33db101193],94262,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91218491-752c-4fa9-8fc6-3afc1c6eff76,"[ae95cd79-acfd-4f2d-861c-0ca749d12ccc, 2569aa6...",-1,12,"[france, ussr]",-1,7.78,-1,13200000,MP_MOVIE,Узник замка Иф,uznik-zamka-if,RELEASE,1989,-1,729598,"[Drama, Adventure, d7214feb-8c11-4aea-aabb-ac9...",[88c67f3a-4595-426b-b70c-3fbdaaf0baa5],42549,okko_ru,2.000000
45fe294a-6cb8-4ebb-80d1-d55a11a34b94,"[c9bbcc35-0e42-4d6c-bb39-986c97202ab5, 1cb898a...",-1,16,[russia],-1,4.83,-1,5640000,MP_MOVIE,Оля + Коля,olja-kolja,RELEASE,2007,-1,749002,[Melodrama],[84ceff16-b9ce-4eb4-8be7-867f5275ace0],432655,NaN,NaN
f7eb3fd9-bd93-457b-be20-b4c07ce5d2e2,"[4cfa7357-8ab8-42fa-9295-83a326710bf4, fdcf2ed...",-1,12,[russia],-1,6.38,-1,10560000,MP_MOVIE,Снова один на всех,snova-odin-na-vsekh,RELEASE,2014,-1,757552,[Melodrama],[f01d9461-0d09-4255-bac5-3ce8c46b2225],783252,okko_ru,2.000000


In [150]:
jf.jaro_winkler_similarity('startrek', '')

0.4671052631578947

In [201]:
data.to_pickle('kp_search_amended_unvalidated.pkl')

In [165]:
data['matched_by'].value_counts()

okko_ru          7907
year_mismatch    1590
alias             528
not found         241
Name: matched_by, dtype: int64

In [163]:
data['matched_by'].isna().sum()

2483

In [277]:
pd.set_option('display.max_rows', 10)
dd = pd.concat(g for _, g in data.groupby("kinopoiskid") if len(g) > 1)
dd[dd['kinopoiskid'] != -1]

,actor,age_restriction_aliases,age_access_type,country,box_office_worldwide,average_rating,box_office_russia,duration,type,name,alias,release_type,release_year,budget,digital_uid,genre,director,kinopoiskid,matched_by,match_coeff
unified_element_uid,,,,,,,,,,,,,,,,,,,,
bae30bd9-fd3b-42a9-91bf-1c775b1e071f,"[7e6681da-52eb-4ea9-aa85-f8c930d27535, 5282990...",-1,16,[russia],-1,7.31,-1,10020000,MOVIE,Казус Кукоцкого,kazus-kukockogo,RELEASE,2005,-1,92160,[Drama],[98cd4053-31df-4f61-8add-6aacdfdef878],162656,okko_ru,2.000000
f3015727-b38c-4912-a91b-52ada4aee440,"[7e6681da-52eb-4ea9-aa85-f8c930d27535, 5282990...",-1,16,[russia],-1,7.31,-1,28800000,SERIAL,Казус Кукоцкого,kazus-kukockogo-99825,RELEASE,2005,-1,99825,[Drama],[98cd4053-31df-4f61-8add-6aacdfdef878],162656,NaN,NaN
dcd9e0a5-7b2d-49df-8b62-dd71b2dfc224,"[4f80a234-8c0f-4d44-940f-71094115c360, a827461...",-1,12,[usa],-1,7.26,-1,7560000,MOVIE,Бриолин: Мюзикл,grease-live,RELEASE,2016,-1,68243,[Drama],[b7deff0b-7a20-4919-831c-07c16ff5fffa],184432,okko_ru,1.091388
3796fb3d-e585-47c3-8154-ff176dc337e1,"[5596c5be-1ff0-446c-8a3f-45a090e8ce7f, a827461...",[CR_9_13],12,[usa],-1,7.06,-1,5880000,MOVIE,Классный мюзикл,high-school-musical,RELEASE,2006,-1,14505,"[Comedy, Melodrama, Drama, Family, Music]",[060c170b-f6d6-45af-a7cf-8e62bb1f83f1],184432,okko_ru,2.000000
a841de97-2683-4a09-9a9e-fe0c2aed358f,"[6eac8cd7-ea76-4552-b82e-9f8ad29de725, ca5a267...",-1,18,[usa],-1,6.00,-1,5640000,MOVIE,Смертные приговоры,death-sentences,RELEASE,2020,-1,52428010,[Thriller],[6eac8cd7-ea76-4552-b82e-9f8ad29de725],256164,okko_ru,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d187943a-0c1a-4573-a8c3-f55c2e878b6b,"[fd401c6b-3e00-4f7f-aea7-5512df6b91e0, 6c9aba4...",-1,18,"[usa, canada]",57223,4.20,42878,5400000,MOVIE,Осада,american-siege,RELEASE,2021,-1,249164132,"[Action, Thriller]",[bb63e20f-702c-4d49-b18f-7f3da8164244],2000707,okko_ru,2.000000
205bbd39-fb08-40dd-ae1e-ebdb52a40b31,"[b79e9608-4f1a-40f7-a3aa-0a0e35d16a1d, 47e5e78...",-1,16,"[usa, great-britain, sweden]",-1,6.99,-1,6000000,MOVIE,Жена,the-wife,RELEASE,2017,-1,108871,[Drama],[d26bb3b1-fc22-4283-b070-dbc1fec2a1f9],4403001,okko_ru,2.000000
7becd8d8-1fb8-4ee6-b903-47817686ddd8,"[6a12c51c-feab-4273-b475-5a66cc8544b9, 0fc279c...",-1,18,[usa],-1,7.60,-1,16620000,SERIAL,Жена путешественника во времени,time-travelers-wife,RELEASE,2022,-1,344754294,"[Sci-Fi, Melodrama, Drama, 9fa28b61-a257-4a3e-...",[af4d065c-ab7f-4d88-a0eb-cb4d669f7a1c],4403001,NaN,NaN


In [275]:
manual_index = "ae494ecb-bb18-47fa-8483-3764e234a633"
manual_kpid = 35526

data.at[manual_index, 'kinopoiskid'] = manual_kpid
data.at[manual_index, 'matched_by'] = 'manual'
data.at[manual_index, 'match_coeff'] = 2.0

In [33]:
data.to_pickle('full_data_unvalidated.pkl')

In [27]:
from kinopoisk_unofficial.request.films.film_request import FilmRequest
from kinopoisk_unofficial.request.films.distributions_request import DistributionsRequest
from kinopoisk_unofficial.request.films.box_office_request import BoxOfficeRequest
from kinopoisk_unofficial.request.staff.staff_request import StaffRequest


def unpack_film_data(film):
    film['genres'] = [entry.genre for entry in film['genres']]
    film['countries'] = [entry.country for entry in film['countries']]
    film['type'] = film['type'].value
    film['production_status'] = film['production_status'].value if film['production_status'] else None
    
    return film

def unpack_dists_data(dists):
    result = []
    for dist in dists.items:
        dist = dist.__dict__
        dist['type'] = dist['type'].value
        dist['sub_type'] = dist['sub_type'].value if dist['sub_type'] else None
        dist['date'] = dist['date'].isoformat() if dist['date'] else None
        dist['country'] = dist['country'].country if dist['country'] else None
        dist['companies'] = [entry.name for entry in dist['companies']]
        
        result.append(dist)
    
    return result

def unpack_staff_data(staff):
    result = []
    for person in staff.items:
        person = person.__dict__
        person['profession_key'] = person['profession_key'].value
        
        result.append(person)
        
    return result


def full_data_parse(data):
    counter = 0
    for index, row in data[data['full_info'].isna()].iterrows():
        kpid = row['kinopoiskid']
        if kpid != -1:
            #print(kpid)
            film_request = FilmRequest(kpid)
            dist_request = DistributionsRequest(kpid)
            box_request = BoxOfficeRequest(kpid)
            
        
            info = api_client.films.send_film_request(film_request).film.__dict__
            #print(info)
            info = unpack_film_data(info)
            
            dists = api_client.films.send_distributions_request(dist_request)
            info['distributions'] = unpack_dists_data(dists)
            
            box = api_client.films.send_box_office_request(box_request)
            info['box_office'] = [entry.__dict__ for entry in box.items]
            
            try:
                staff_request = StaffRequest(kpid)
                staff = api_client.staff.send_staff_request(staff_request)
                info['staff'] = unpack_staff_data(staff)
            except Exception:
                info['staff'] = None
                pass
            
            
            data.at[index, 'full_info'] = json.dumps(info)

        
        counter += 1
        if counter % 100 == 0:
            print(f'Processing record {counter}')

In [31]:
full_data_parse(data)

Processing record 100
Processing record 200
Processing record 300
Processing record 400
Processing record 500
Processing record 600
Processing record 700
Processing record 800
Processing record 900
Processing record 1000
Processing record 1100
Processing record 1200
Processing record 1300
Processing record 1400
Processing record 1500
Processing record 1600
Processing record 1700
Processing record 1800
Processing record 1900
Processing record 2000
Processing record 2100
Processing record 2200
Processing record 2300
Processing record 2400
Processing record 2500
Processing record 2600
Processing record 2700
Processing record 2800
Processing record 2900
Processing record 3000
Processing record 3100
Processing record 3200
Processing record 3300
Processing record 3400
Processing record 3500
Processing record 3600
Processing record 3700
Processing record 3800
Processing record 3900
Processing record 4000
Processing record 4100
Processing record 4200
Processing record 4300
Processing record 44

In [8]:
data

,actor,age_restriction_aliases,age_access_type,country,box_office_worldwide,average_rating,box_office_russia,duration,type,name,...,release_type,release_year,budget,digital_uid,genre,director,kinopoiskid,matched_by,match_coeff,full_info
unified_element_uid,,,,,,,,,,,,,,,,,,,,,
e785baa6-f175-42b4-9e16-4319ac7991d5,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, 21c7a45...",[CR_9_13],12,[usa],-1,6.21,-1,6420000,MOVIE,Звёздный путь 5: Последний рубеж,...,RELEASE,1989,27800000,2870,"[Sci-Fi, Action, Thriller, Adventure]",[cff3362f-ad91-498d-80b3-ea3ab8ec65cf],6580,okko_ru,2.000000,"{""kinopoisk_id"": 6580, ""imdb_id"": ""tt0098382"",..."
4593737e-de9c-40df-97db-fb3cf85a08ef,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, 21c7a45...",-1,16,[usa],-1,7.20,-1,6780000,MOVIE,Звёздный путь 6: Неоткрытая страна,...,RELEASE,1991,30000000,2873,"[Sci-Fi, Action, Thriller, Adventure]",[bab6b7f4-4506-458f-9091-d567505674f2],6460,okko_ru,2.000000,"{""kinopoisk_id"": 6460, ""imdb_id"": ""tt0102975"",..."
11ba66db-e941-4c3a-8da6-d8900e56f8c7,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, bcad17e...",-1,18,[usa],-1,4.00,-1,6000000,MOVIE,В поисках древнего артефакта,...,RELEASE,2019,-1,52428009,[Horror],[895f5774-964a-4c5a-ae22-d697d3e3e620],1228112,alias,1.538957,"{""kinopoisk_id"": 1228112, ""imdb_id"": ""tt946984..."
3f30a2ef-53b7-40e3-954f-1bdfc38a6d17,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, 30dd19c...",-1,18,[usa],-1,6.99,-1,3780000,MOVIE,Прожарка Чарли Шина,...,RELEASE,2011,-1,1084933,"[Comedy, Documentary]",[1de22aff-430e-4af9-bf1d-159dbf8e9269],623934,okko_ru,2.000000,"{""kinopoisk_id"": 623934, ""imdb_id"": ""tt1985970..."
cdfa700f-122d-41e5-b8dc-9c6813bab6d2,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, a805fdd...",-1,18,[usa],-1,6.00,-1,3780000,MOVIE,Прожарка Уильяма Шэтнера,...,RELEASE,2006,-1,1191553,[Comedy],[1de22aff-430e-4af9-bf1d-159dbf8e9269],893637,okko_ru,2.000000,"{""kinopoisk_id"": 893637, ""imdb_id"": ""tt0840305..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ac0c9152-c070-43a0-bb99-7e72764b7c0a,[d08bba89-e937-40f3-b2a7-7ea26de4c246],"[CR_0_4, CR_5_8]",0,[brasil],-1,-1.00,-1,5400000,SERIAL,Изобретая игры,...,RELEASE,2019,-1,388072866,[287a1485-7a88-4c2f-bc94-ca418b6c47a1],[ecc88cca-7660-4d74-a0af-18ca7069a743],4950630,okko_ru,2.000000,NaN
ce8d2906-a33f-4140-9ffe-e091990d71a4,[50ed4519-ad24-47b3-b135-947dba796372],"[CR_5_8, CR_9_13]",6,[russia],-1,-1.00,-1,3600000,SERIAL,Большие игры,...,RELEASE,2022,-1,376604149,"[ForKids, 1f22ccf1-288a-4e6e-b39a-7502799e7125]",[ad784ddb-a602-4082-bf6f-f83ec5a96edf],976636,okko_ru,2.000000,NaN
0bfb5a13-50fc-48f3-b51a-a30971024651,"[d658fbcd-8e80-4880-a25c-e7a209998969, b6fe1a3...","[CR_0_4, CR_0_13, CR_5_8, CR_9_13]",0,[china],-1,-1.00,-1,40560000,SERIAL,Природный патруль,...,RELEASE,2021,-1,375536499,[287a1485-7a88-4c2f-bc94-ca418b6c47a1],[9d6f5496-2d1b-4f21-a2d8-52b316295c4a],5086549,okko_ru,2.000000,NaN


In [32]:
data.loc[(data['full_info'].isna()) & (data['kinopoiskid'] != -1)]

,actor,age_restriction_aliases,age_access_type,country,box_office_worldwide,average_rating,box_office_russia,duration,type,name,...,release_type,release_year,budget,digital_uid,genre,director,kinopoiskid,matched_by,match_coeff,full_info
unified_element_uid,,,,,,,,,,,,,,,,,,,,,
